In [1]:
import sys

sys.path.insert(0, '/home/workspace/notebook')

from utils import *

paths = "/home/workspace/notebook/data/GSE249005_bone_marrow_organoids"

glob.glob(os.path.join(paths, "*.gz"))

['/home/workspace/notebook/data/GSE249005_bone_marrow_organoids/GSM7924557_BMO5_features.tsv.gz',
 '/home/workspace/notebook/data/GSE249005_bone_marrow_organoids/GSM7924553_BMO1_features.tsv.gz',
 '/home/workspace/notebook/data/GSE249005_bone_marrow_organoids/GSM7924556_BMO4_features.tsv.gz',
 '/home/workspace/notebook/data/GSE249005_bone_marrow_organoids/GSM7924556_BMO4_matrix.mtx.gz',
 '/home/workspace/notebook/data/GSE249005_bone_marrow_organoids/GSM7924556_BMO4_barcodes.tsv.gz',
 '/home/workspace/notebook/data/GSE249005_bone_marrow_organoids/GSM7924557_BMO5_barcodes.tsv.gz',
 '/home/workspace/notebook/data/GSE249005_bone_marrow_organoids/GSM7924554_BMO2_barcodes.tsv.gz',
 '/home/workspace/notebook/data/GSE249005_bone_marrow_organoids/GSM7924554_BMO2_matrix.mtx.gz',
 '/home/workspace/notebook/data/GSE249005_bone_marrow_organoids/GSM7924555_BMO3_matrix.mtx.gz',
 '/home/workspace/notebook/data/GSE249005_bone_marrow_organoids/GSM7924557_BMO5_matrix.mtx.gz',
 '/home/workspace/notebook/d

In [2]:
adata1 = sc.read_10x_mtx(paths, prefix='GSM7924553_BMO1_')
adata2 = sc.read_10x_mtx(paths, prefix='GSM7924554_BMO2_')
adata3 = sc.read_10x_mtx(paths, prefix='GSM7924555_BMO3_')
adata4 = sc.read_10x_mtx(paths, prefix='GSM7924556_BMO4_')
adata5 = sc.read_10x_mtx(paths, prefix='GSM7924557_BMO5_')

labels = [f'BMO{n}' for n in range(1, 6)]

adatas = [adata1, adata2, adata3, adata4, adata5]

for adata, label in zip(adatas, labels):
    adata.obs['id'] = label

    adata.var["mt"] = adata.var_names.str.startswith("MT-")
    adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
    adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")
    adata.var["ig"] = adata.var_names.str.startswith("IG")

    sc.pp.calculate_qc_metrics(adata, qc_vars=["mt", "ribo", "hb", "ig"], inplace=True, percent_top=[20], log1p=True)

    adata.obs["outlier"] = (
        is_outlier(adata, "log1p_total_counts", 5)                # calling outlier cells with log1p_total_counts greater than 5 MAD's
        | is_outlier(adata, "log1p_n_genes_by_counts", 5)         # calling outlier cells with log1p_n_genes_by_counts greater than 5 MAD's
        | is_outlier(adata, "pct_counts_in_top_20_genes", 5)      # calling outlier cells with pct_counts_in_top_20_genes greater than 5 MAD's
        | is_outlier(adata, "pct_counts_mt", 5)
        | is_outlier(adata, "pct_counts_ribo", 5)
        | is_outlier(adata, "pct_counts_hb", 5)
        | is_outlier(adata, "pct_counts_ig", 5)
    )
    
    print(adata)

# adata = ad.concat(adatas, join='outer', merge='same')

AnnData object with n_obs × n_vars = 5014 × 20639
    obs: 'id', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'total_counts_ig', 'log1p_total_counts_ig', 'pct_counts_ig', 'outlier'
    var: 'gene_ids', 'feature_types', 'mt', 'ribo', 'hb', 'ig', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
AnnData object with n_obs × n_vars = 9369 × 20639
    obs: 'id', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'total

In [ ]:
for adata in adatas:
    adata.var["mt"] = adata.var_names.str.startswith("MT-")
    adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
    adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")
    adata.var["ig"] = adata.var_names.str.startswith("IG")

    sc.pp.calculate_qc_metrics(
        adata, qc_vars=["mt", "ribo", "hb", "ig"], inplace=True, log1p=True
    )

    # sc.pl.violin(adata, "pct_counts_mt")
    # sc.pl.violin(adata, "pct_counts_ig")

In [ ]:
for adata in adatas:
    adata.layers["counts"] = adata.X.copy()
    sc.pp.normalize_total(adata)
    sc.pp.log1p(adata)
    sc.tl.pca(adata)
    sc.pp.neighbors(adata)
    sc.tl.umap(adata)
    sc.tl.leiden(adata, resolution=1, flavor="igraph", n_iterations=4)

    sc.pl.umap(adata, color=["leiden"])